### Ingests camera metadata into a Postgres database

In [1]:
from pathlib import Path

import polars as pl

from afft.filesystem import search_directory
from afft.io.sql import Endpoint, create_endpoint, write_database

from afft.utils.log import logger
from afft.utils.result import Ok, Err, Result


def ingest_cameras_database(
    endpoint: Endpoint, data_frames: dict[str, pl.DataFrame]
) -> None:
    """Ingests a collection of camera data frames into a database."""

    insert_results: dict[str, Result] = {
        table: write_database(endpoint, table, data_frame, if_table_exists="replace")
        for table, data_frame in data_frames.items()
    }

    for name, insert_result in insert_results.items():
        match insert_result:
            case Ok(None):
                pass
            case Err(message):
                logger.error(message)


def main() -> None:
    """Main function."""

    DATABASE: str = "acfr_auv_cameras"
    HOST: str = "localhost"
    PORT: int = 5432
    DIRECTORY: Path = Path("/data/kingston_snv_01/acfr_cameras_metadata/")

    # Query directory for camera files
    files: list[Path] = search_directory(DIRECTORY, "*_cameras_metadata.csv").unwrap()

    # Read data frames and label with path stem
    data_frames: dict[str, pl.DataFrame] = dict()
    for path in files:
        label: str = path.stem
        data_frame: pl.DataFrame = pl.read_csv(source=path)

        assert len(data_frame) > 0, f"no cameras in {path.name}"

        data_frames[label] = data_frame

    logger.info("")
    logger.info(f"Base directory: {DIRECTORY}")
    logger.info(f"Camera files: {len(files)}")
    logger.info("")

    match create_endpoint(database=DATABASE, host=HOST, port=PORT):
        case Ok(endpoint):
            ingest_cameras_database(endpoint, data_frames)
        case Err(message):
            logger.error(message)


# Invoke main
main()

2024-10-16 18:17:11.740 | INFO     | __main__:main:51 - 
2024-10-16 18:17:11.740 | INFO     | __main__:main:52 - Base directory: /data/kingston_snv_01/acfr_cameras_metadata
2024-10-16 18:17:11.741 | INFO     | __main__:main:53 - Camera files: 52
2024-10-16 18:17:11.741 | INFO     | __main__:main:54 - 
